In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import os,sys
import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import random
import math
from collections import Counter

In [2]:
with open('data/stop_words.txt') as f:
    stop_words_list = f.read().splitlines() #stop_list1
stop_list2 = pickle.load(open('data/stop_list2.pkl','rb'))
ps = PorterStemmer() # Stemming
stop_words = set(stopwords.words('english')) #Stopword
short = ['.', ',', '"', "\'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "'at",
         "_","`","\'\'","--","``",".,","//",":","___",'_the','-',"'em",".com",
                   '\'s','\'m','\'re','\'ll','\'d','n\'t','shan\'t',"...","\'ve",'u']
stop_words_list.extend(short)
stop_words_list.extend(stop_list2)
stop_words.update(stop_words_list)

In [3]:
def preprocess(texts):
    tokens = [i for i in word_tokenize(texts.lower()) if i not in stop_words]  # Tokenization.# Lowercasing
    token_result = ''
    token_result_ = ''
    for i,token in enumerate(tokens): #list2str
        token_result += ps.stem(token) + ' '
    token_result = ''.join([i for i in token_result if not i.isdigit()])
    token_result = [i for i in word_tokenize(token_result) if i not in stop_words]
    for i,token in enumerate(token_result):
        token_result_ += token + ' '
    return token_result_

## Feature Selection
* tf-idf
* chi-square
* likelihood
* PMI
* EMI

=> build dictionary in 500 words

### LLR

In [95]:
dict_df = pd.read_csv('../HW2/output/dictionary.txt',header=None,index_col=None,sep=' ')
terms = dict_df[1].tolist() #all terms


In [96]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
train_dict #class:doc_id

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16'],
 '3': ['813',
  '817',
  '818',
  '819',
  '820',
  '821',
  '822',
  '824',
  '825',
  '826',
  '828',
  '829',
  '830',
  '832',
  '833'],
 '4': ['635',
  '680',
  '683',
  '702',
  '704',
  '705',
  '706',
  '708',
  '709',
  '719',
  '720',
  '722',
  '723',
  '724',
  '726'],
 '5': ['646',
  '751',
  '781',
  '794',
  '798',
  '799',
  '801',
  '812',
  '815',
  '823',
  '831',
  '839',
  '840',
  '841',
  '842'],
 '6': ['995',
  '998',
  '999',
  '1003',
  '1005',
  '1006',
  '1007',
  '1009',
  '1011',
  '1012',
  '1013',
  '1014',
  '1015',
  '1016',
  '1019'],
 '7': ['700',
  '730',
  '731',
  '732',
  '733',
  '735',
  '740',
  '744',
  '752',
  '754',
  '755',
  '756',
  '757',
  '759',
  '760'],
 '8': ['262',
  '296',
  '30

In [97]:
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
for c,d in train_dict.items():
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(set(filter(None,tokens_all)))
        class_token.append(tokens_all)
    class_token_dict[c]=class_token
    class_token=[]
len(class_token_dict['1'])

15

In [71]:
# train_dict = {}
# for c,t in class_token_dict.items():
#     train_dict[c] = list(set(t))
# #     train_dict[c] = dict(Counter(t))
# train_dict

TypeError: unhashable type: 'list'

In [98]:
dict_df.drop(0,axis=1,inplace=True)
dict_df.columns = ['term','score']
dict_df.index = dict_df['term']
dict_df.drop('term',axis=1,inplace=True)
dict_df

,score
term,
abandon,39
abc,51
abcnews.com,3
abdallah,2
abdel,3
abdomin,2
abduct,2
abdul,53
abdul-karim,11


In [147]:
dict_df['score'] = 0
dict_df['score_chi'] = 0
dict_df['score_emi'] = 0
# c=1
for term in tqdm(terms): #each term
    scores = []
    scores_chi = []
    scores_emi = []
    c=1
    for _ in range(len(class_token_dict)): # each class
        n11=e11=m11=0
        n10=e10=m10=0
        n01=e01=m01=0
        n00=e00=m00=0
        for k,v in class_token_dict.items():
#             print(k,c)
            if k == str(c): #ontopic
                for r in v:
                    if term in r:
                        n11+=1
                    else:
                        n10+=1
#                 c+=1
            else: #off topic
                for r in v:
                    if term in r:
                        n01+=1
                    else:
                        n00+=1
#                 c+=1
        c+=1
        n11+=1e-8
        n10+=1e-8
        n01+=1e-8
        n00+=1e-8
        N = n11+n10+n01+n00
        e11 = N * (n11+n01)/N * (n11+n10)/N #chi-squre
        e10 = N * (n11+n10)/N * (n10+n00)/N
        e01 = N * (n11+n01)/N * (n01+n00)/N
        e00 = N * (n01+n00)/N * (n10+n00)/N
        score_chi = ((n11-e11)**2)/e11 + ((n10-e10)**2)/e10 + ((n01-e01)**2)/e01 + ((n00-e00)**2)/e00
        
        n11 = n11 - 1e-8 + 1e-6
        n10 = n10 - 1e-8 + 1e-6
        n01 = n01 - 1e-8 + 1e-6
        n00 = n00 - 1e-8 + 1e-6
        N = n11+n10+n01+n00
        m11 = (n11/N) * math.log(((n11/N)/((n11+n01)/N * (n11+n10)/N)),2) #EMI
        m10 = n10/N * math.log((n10/N)/((n11+n10)/N * (n10+n00)/N),2)
        m01 = n01/N * math.log((n01/N)/((n11+n01)/N * (n01+n00)/N),2)
        m00 = n00/N * math.log((n00/N)/((n01+n00)/N * (n10+n00)/N),2)
        score_emi = m11 + m10 + m01 + m00
        
#         print(n11,n10,n01,n00)
        n11-=1e-6
        m10-=1e-6
        n01-=1e-6
        n00-=1e-6
        N = n11+n10+n01+n00
        score = (((n11+n01)/N) ** n11) * ((1 - ((n11+n01)/N)) ** n10) * (((n11+n01)/N) ** n01) * ((1 - ((n11+n01)/N)) ** n00)
        score /= ((n11/(n11+n10)) ** n11) * ((1 - (n11/(n11+n10))) ** n10) * ((n01/(n01+n00)) ** n01) * ((1 - (n01/(n01+n00))) ** n00)
        score = -2 * math.log(score, 10) #LLR
        scores.append(score)
        scores_chi.append(score_chi)
        scores_emi.append(score_emi)
#         c+=1
    dict_df.loc[term,'score'] = np.mean(scores)
    dict_df.loc[term,'score_chi'] = np.mean(scores_chi)
    dict_df.loc[term,'score_emi'] = np.mean(scores_emi)
dict_df    

100%|██████████| 8913/8913 [00:45<00:00, 194.55it/s]


,score,score_chi,score_emi
term,,,
abandon,0.468414,2.197541,3.989836e-03
abc,0.337624,1.423542,2.875803e-03
abcnews.com,0.000000,4.444311,9.312376e-11
abdallah,0.480440,3.081259,4.092274e-03
abdel,0.000000,4.444311,9.312376e-11
abdomin,0.000000,4.444311,9.312376e-11
abduct,0.000000,4.444311,9.312376e-11
abdul,0.000000,4.444311,9.312376e-11
abdul-karim,0.000000,4.444311,9.312376e-11


In [202]:
dict_df2 = pd.read_csv('../HW2/output/dictionary.txt',header=None,index_col=None,sep=' ')
dict_df2.columns = ['id','term','freq']
dict_df2['sum'] = 0.0
dict_df2

,id,term,freq,sum
0,1,abandon,39,0.0
1,2,abc,51,0.0
2,3,abcnews.com,3,0.0
3,4,abdallah,2,0.0
4,5,abdel,3,0.0
5,6,abdomin,2,0.0
6,7,abduct,2,0.0
7,8,abdul,53,0.0
8,9,abdul-karim,11,0.0
9,10,abdullah,40,0.0


In [204]:
tf_list = next(os.walk('../HW2/output/tf-idf/'))[2]
# df_list = [dict_df2]
for tf in tf_list:
#     print(tf)
    df2 = pd.read_csv('../HW2/output/tf-idf/'+tf,header=None,index_col=None,sep=' ',skiprows=[0])
    df2.columns = ['id','tfidf']
    df3 = pd.merge(dict_df2,df2,on='id',how='outer')
    df3.fillna(0,inplace=True)
    dict_df2['sum']+=df3['tfidf']
#     break
#     df_list.append(df2)
# df3 = pd.concat(df_list).groupby(level=0).sum()
# df3 = pd.concat([df3,df2]).groupby(level=0).sum()
# df3 = pd.merge(dict_df2,df2,on='id',how='outer')
# df3 = pd.merge(df3,df2,on='id',how='outer')?
# df3[df3.id == 2]
dict_df2['avg_tfidf'] = dict_df2['sum']/dict_df2['freq']
dict_df2 = dict_df2.drop(['freq','sum'],axis=1)
dict_df2

,id,term,avg_tfidf
0,1,abandon,0.010747
1,2,abc,0.031715
2,3,abcnews.com,0.067393
3,4,abdallah,0.013193
4,5,abdel,0.007849
5,6,abdomin,0.021898
6,7,abduct,0.032520
7,8,abdul,0.001701
8,9,abdul-karim,0.010270
9,10,abdullah,0.013616


In [209]:
dict_df['term'] = dict_df.index
dict_df3 = pd.merge(dict_df,dict_df2,on='term',how='outer')
dict_df3

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: FutureWarning: 'term' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,score,score_chi,score_emi,term,id,avg_tfidf
0,0.468414,2.197541,3.989836e-03,abandon,1,0.010747
1,0.337624,1.423542,2.875803e-03,abc,2,0.031715
2,0.000000,4.444311,9.312376e-11,abcnews.com,3,0.067393
3,0.480440,3.081259,4.092274e-03,abdallah,4,0.013193
4,0.000000,4.444311,9.312376e-11,abdel,5,0.007849
5,0.000000,4.444311,9.312376e-11,abdomin,6,0.021898
6,0.000000,4.444311,9.312376e-11,abduct,7,0.032520
7,0.000000,4.444311,9.312376e-11,abdul,8,0.001701
8,0.000000,4.444311,9.312376e-11,abdul-karim,9,0.010270
9,0.000000,4.444311,9.312376e-11,abdullah,10,0.013616


In [210]:
cols = list(dict_df3)
cols[4], cols[3], cols[5], cols[1], cols[0], cols[2] = cols[0], cols[1] , cols[2] , cols[3], cols[4], cols[5]
dict_df3 = dict_df3.ix[:,cols]
dict_df3

/home/leoqaz12/.local/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


,id,term,avg_tfidf,score_chi,score,score_emi
0,1,abandon,0.010747,2.197541,0.468414,3.989836e-03
1,2,abc,0.031715,1.423542,0.337624,2.875803e-03
2,3,abcnews.com,0.067393,4.444311,0.000000,9.312376e-11
3,4,abdallah,0.013193,3.081259,0.480440,4.092274e-03
4,5,abdel,0.007849,4.444311,0.000000,9.312376e-11
5,6,abdomin,0.021898,4.444311,0.000000,9.312376e-11
6,7,abduct,0.032520,4.444311,0.000000,9.312376e-11
7,8,abdul,0.001701,4.444311,0.000000,9.312376e-11
8,9,abdul-karim,0.010270,4.444311,0.000000,9.312376e-11
9,10,abdullah,0.013616,4.444311,0.000000,9.312376e-11


In [211]:
dict_df3.columns = ['id','term','avg_tfidf','score_chi','score_llr','score_emi']
dict_df3.to_csv('output/feature_selection_df.csv')

### select top 500
* 取各col的mean+1.45*std
* 再去做投票，超過兩票的流下來看剩下哪幾個

In [4]:
dict_df3 = pd.read_csv('output/feature_selection_df.csv',index_col=None)
threshold_tfidf = np.mean(dict_df3['avg_tfidf'])+2*np.std(dict_df3['avg_tfidf']) #1.45=>502
threshold_chi = np.mean(dict_df3['score_chi'])+2*np.std(dict_df3['score_chi']) #1=>350
threshold_llr = np.mean(dict_df3['score_llr'])+2*np.std(dict_df3['score_llr']) #1.75=>543
threshold_emi = np.mean(dict_df3['score_emi'])+2*np.std(dict_df3['score_emi']) #1.75=>543

print('avg_tfidf',threshold_tfidf)
# dict_df3[dict_df3.score_llr>0.1]

avg_tfidf 0.04284210930851971


In [5]:
df1 = dict_df3[dict_df3['avg_tfidf']>threshold_tfidf]
df2 = dict_df3[dict_df3['score_chi']>threshold_chi]
df3 = dict_df3[dict_df3['score_llr']>threshold_llr]
df4 = dict_df3[dict_df3['score_emi']>threshold_emi]
df_vote = dict_df3
df_vote['vote']=0
df_vote

,Unnamed: 0,id,term,avg_tfidf,score_chi,score_llr,score_emi,vote
0,0,1,abandon,0.010747,2.197541,0.468414,3.989836e-03,0
1,1,2,abc,0.031715,1.423542,0.337624,2.875803e-03,0
2,2,3,abcnews.com,0.067393,4.444311,0.000000,9.312376e-11,0
3,3,4,abdallah,0.013193,3.081259,0.480440,4.092274e-03,0
4,4,5,abdel,0.007849,4.444311,0.000000,9.312376e-11,0
5,5,6,abdomin,0.021898,4.444311,0.000000,9.312376e-11,0
6,6,7,abduct,0.032520,4.444311,0.000000,9.312376e-11,0
7,7,8,abdul,0.001701,4.444311,0.000000,9.312376e-11,0
8,8,9,abdul-karim,0.010270,4.444311,0.000000,9.312376e-11,0
9,9,10,abdullah,0.013616,4.444311,0.000000,9.312376e-11,0


In [7]:
df2

,Unnamed: 0,id,term,avg_tfidf,score_chi,score_llr,score_emi,vote
12,12,13,abidjan,0.035318,8.688017,2.656985,0.022632,0
81,81,82,aden,0.015046,7.781461,2.354213,0.020053,0
130,130,131,african,0.014446,9.620281,2.973231,0.025325,0
136,136,137,agassi,0.066567,8.688017,2.656985,0.022632,0
140,140,141,agent,0.018818,7.038590,2.536942,0.021609,0
202,202,203,alassan,0.018521,9.620281,2.973231,0.025325,0
208,208,209,alberto,0.020363,13.557326,4.536663,0.038642,0
310,310,311,andr,0.013081,8.688017,2.656985,0.022632,0
405,405,406,arabian,0.014715,6.265580,1.877979,0.015996,0
473,473,474,ashcroft,0.036735,7.781461,2.354213,0.020053,0


In [8]:
df_vote.loc[df1.id-1,'vote'] += 1
df_vote.loc[df2.id-1,'vote'] += 1
df_vote.loc[df3.id-1,'vote'] += 1
df_vote.loc[df4.id-1,'vote'] += 1
df_vote

,Unnamed: 0,id,term,avg_tfidf,score_chi,score_llr,score_emi,vote
0,0,1,abandon,0.010747,2.197541,0.468414,3.989836e-03,0
1,1,2,abc,0.031715,1.423542,0.337624,2.875803e-03,0
2,2,3,abcnews.com,0.067393,4.444311,0.000000,9.312376e-11,1
3,3,4,abdallah,0.013193,3.081259,0.480440,4.092274e-03,0
4,4,5,abdel,0.007849,4.444311,0.000000,9.312376e-11,0
5,5,6,abdomin,0.021898,4.444311,0.000000,9.312376e-11,0
6,6,7,abduct,0.032520,4.444311,0.000000,9.312376e-11,0
7,7,8,abdul,0.001701,4.444311,0.000000,9.312376e-11,0
8,8,9,abdul-karim,0.010270,4.444311,0.000000,9.312376e-11,0
9,9,10,abdullah,0.013616,4.444311,0.000000,9.312376e-11,0


In [10]:
df_vote = df_vote[df_vote.vote>1] #(1,2)=>375 #(1,1)=>422
df_vote = df_vote.filter(['id','term','vote'])
df_vote

In [11]:
df_vote.to_csv('output/500terms_df.csv')

## Classifier
* 7-fold
* MNB
* BNB
* self-train / co-train
* ens voting (BNB lower weight)
* auto-skleran / auto-keras

REF: http://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html

In [4]:
df_vote = pd.read_csv('output/500terms_df.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))

train_X = []
train_Y = []
len(trainX)

422

In [115]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
train_X = []
train_Y= []
train_ids = []
for c,d in tqdm(train_dict.items()):
    for doc in d:
        train_ids.append(doc)
        trainX = np.array([0]*len(terms_li))
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
        tokens_all = dict(Counter(tokens_all))
        for key,value in tokens_all.items():
            if key in terms_li:
                trainX[terms_li.index(key)] = int(value)
#         trainX = np.array(trainX)
        
#         for token in tokens_all:
#             if token in terms_li:
#                 ind = terms_li.index(token)
#                 trainX[ind]+=1
        train_X.append(trainX)
        train_Y.append(int(c))
        
train_X = np.array(train_X)
train_Y = np.array(train_Y)

#         tokens_all = list(set(filter(None,tokens_all)))
#         class_token.append(tokens_all)
#     class_token_dict[c]=class_token
#     class_token=[]
# len(class_token_dict['1'])

100%|██████████| 13/13 [00:01<00:00,  9.57it/s]


In [106]:
train_X.shape , train_Y.shape

((195, 422), (195,))

In [119]:
terms_li

['espionag',
 'lindsay',
 'explos',
 'accus',
 'suprem',
 'open',
 'thursday',
 'yugoslav',
 'middl',
 'russia',
 'georg',
 'trade',
 'men',
 'governor',
 'buri',
 'yemen',
 'texa',
 'vojislav',
 'defend',
 'ballot',
 'violenc',
 'roger',
 'alert',
 'court',
 'kostunica',
 'caus',
 'appear',
 'year',
 'die',
 'dead',
 'debat',
 'american',
 'gbagbo',
 'peruvian',
 'nuclear',
 'survivor',
 'jack',
 'economi',
 'govern',
 'defens',
 'congress',
 'neighborhood',
 'weather',
 'polit',
 'cooper',
 'citi',
 'buy',
 'bare',
 'agreement',
 'mile',
 'sub',
 'parti',
 'hospit',
 'hit',
 'coup',
 'australian',
 'abidjan',
 'foreign',
 'power',
 'medic',
 'twin-engin',
 'cessna',
 'gen.',
 'salvador',
 'trial',
 'loui',
 'support',
 'moscow',
 'peac',
 'belgrad',
 'asylum',
 'chi',
 'robert',
 'hous',
 'west',
 'guei',
 'defeat',
 'radioprograma',
 'dalla',
 'terror',
 'resign',
 'opposit',
 'barent',
 'play',
 'guatemala',
 'uss',
 'beat',
 'activist',
 'torpedo',
 'damag',
 'electr',
 'win',
 'v

In [120]:
#建立term index matrix
tokens_all_class=[]
term_tf_mat=[]
for c,d in tqdm(train_dict.items()):
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(filter(None,tokens_all))
        tokens_all_class.extend(tokens_all)
    tokens_all = dict(Counter(tokens_all_class))
    term_tf_mat.append(tokens_all)

100%|██████████| 13/13 [00:01<00:00, 10.28it/s]


In [148]:
term_tf_mat[13]

IndexError: list index out of range

In [179]:
def train_MNB(train_set=train_dict,term_list=terms_li,term_tf_mat=term_tf_mat):
    prior = np.zeros(len(train_set))
    cond_prob = np.zeros((len(train_set), len(term_list)))
    
    for i,docs in train_set.items(): #13 classes 1~13
        prior[int(i)-1] = len(docs)/len(train_ids) #那個類別的文章有幾個/總共的文章數目 0~12
        token_count=0
        class_tf = np.zeros(len(term_list))
        for idx,term in enumerate(term_list):
            try:
                class_tf[idx] = term_tf_mat[int(i)-1][term]  #term在class的出現次數
            except:
                token_count+=1

        class_tf = class_tf + 1e-8#+ np.ones(len(term_list)) #smoothing (可改)
        class_tf = class_tf/(sum(class_tf) +token_count) #該class總共的token數(可改)
        cond_prob[int(i)-1] = class_tf #0~12
    return prior, cond_prob

In [180]:
prior,cond_prob = train_MNB()

In [144]:
# cond_probtok


array([[3.12000890e+02, 3.12000890e+02, 3.12013345e+02, ...,
        3.12000890e+02, 3.12000890e+02, 3.12000890e+02],
       [2.22000400e+02, 2.22000400e+02, 2.22006805e+02, ...,
        2.22008006e+02, 2.22000400e+02, 2.22000400e+02],
       [1.66000253e+02, 1.66000253e+02, 1.66004558e+02, ...,
        1.66005065e+02, 1.66000253e+02, 1.66000253e+02],
       ...,
       [2.50008861e+01, 2.50005907e+01, 2.50041350e+01, ...,
        2.50016245e+01, 2.50013291e+01, 2.50002215e+01],
       [4.00078844e+00, 4.00052562e+00, 4.00367937e+00, ...,
        4.00144547e+00, 4.00118265e+00, 4.00026281e+00],
       [7.00566291e-04, 4.67044194e-04, 3.26930936e-03, ...,
        1.28437153e-03, 1.05084944e-03, 1.16761049e-03]])

In [132]:
for i in range(13):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [181]:
def predict_MNB(test_id,prob=False,prior=prior,cond_prob=cond_prob,term_list=terms_li):
    f = open('data/IRTM/'+str(test_id)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
    tokens_all = list(filter(None,tokens_all))
    
    class_scores = []
    score = 0
    for i in range(13):
        score += math.log(prior[i],10)
        for token in tokens_all:
            if token in term_list:
                score += math.log(cond_prob[i][term_list.index(token)])
        class_scores.append(score)
    if prob:
        return np.array(class_scores)
    else:
        return(np.argmax(class_scores)+1)

In [185]:
ans = predict_MNB(17,prob=False) # 17 18 20 21
ans

1

In [80]:
def MNB(input_X,input_Y=None,prior_log_class=None,log_prob_feature=None,train=True,prob=False,smooth=1.0):
    if train:
        sample_num = input_X.shape[0]
        match_data = [[x for x, t in zip(input_X, input_Y) if t == c] for c in np.unique(input_Y)]
        prior_log_class = [np.log(len(i) / sample_num) for i in match_data]
        counts = np.array([np.array(i).sum(axis=0) for i in match_data]) + smooth
        log_prob_feature = np.log(counts / counts.sum(axis=1)[np.newaxis].T)
        return prior_log_class,log_prob_feature
    else:
        probability = [(log_prob_feature * x).sum(axis=1) + prior_log_class for x in input_X]
        if prob:
            return probability
        else:
            ans = np.argmax(probability,axis=1)
            return ans

In [108]:
prior , log = MNB(input_X=train_X,input_Y=train_Y,train=True)
# prob = MNB(input_X=X_test,prior_log_class=param1,log_prob_feature=param2,train=False,prob=True)
ans2 = MNB(input_X=test_X,prior_log_class=prior,log_prob_feature=log,train=False,prob=False)
# ans

In [6]:
class MultinomialNB(object):
    def __init__(self, alpha=1.0): #smooting
        self.alpha = alpha

    def fit(self, X, y):
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        self.feature_log_prob_ = np.log(count / count.sum(axis=1)[np.newaxis].T)
        return self

    def predict_log_proba(self, X):
        return [(self.feature_log_prob_ * x).sum(axis=1) + self.class_log_prior_
                for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)

In [18]:
class BernoulliNB(object):
    def __init__(self, alpha=1.0, binarize=0.0):
        self.alpha = alpha
        self.binarize = binarize
    def _binarize_X(self, X):
        return np.where(X > self.binarize, 1, 0) if self.binarize != None else X
    def fit(self, X, y):
        X = self._binarize_X(X)
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        smoothing = 2 * self.alpha
        n_doc = np.array([len(i) + smoothing for i in separated])
        self.feature_prob_ = count / n_doc[np.newaxis].T
        return self

    def predict_log_proba(self, X):
        X = self._binarize_X(X)
        return [(np.log(self.feature_prob_) * x + \
                 np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        X = self._binarize_X(X)
        return np.argmax(self.predict_log_proba(X), axis=1)

In [296]:
X = np.array([
    [2,1,0,0,0,0],
    [2,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,1]
])
y = np.array([0,1,2,3])
nb = MultinomialNB().fit(X, y)
X_test = np.array([[3,0,0,0,1,1],[0,1,1,0,1,1],[1,0,0,0,1,1],[2,1,0,0,0,0],[2,0,1,0,0,0],[1,0,0,1,0,0]])
print(nb.predict_log_proba(X_test))

[array([-9.07658038, -9.07658038, -9.70406053, -8.90668135]), array([-9.48204549, -9.48204549, -9.70406053, -8.78889831]), array([-6.8793558 , -6.8793558 , -6.93147181, -5.89852655]), array([-5.08759634, -5.78074352, -6.23832463, -6.59167373]), array([-5.78074352, -5.08759634, -6.23832463, -6.59167373]), array([-4.68213123, -4.68213123, -4.15888308, -5.08759634])]


### Prediction

In [ ]:
df_vote = pd.read_csv('output/500terms_df.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))
len(terms_li)

In [84]:
trainid

['13',
 '485',
 '520',
 '523',
 '526',
 '527',
 '529',
 '530',
 '531',
 '532',
 '536',
 '537',
 '538',
 '539',
 '540',
 '541']

In [94]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
test_id = []
train_ids=[]
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_ids.extend(trainid[1:])
for i in range(1095):
    if str(i+1) not in train_ids:
        test_id.append(i+1)
#     train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
test_X = []
# train_Y= []
# for c,d in tqdm(train_dict.items()):
for doc in tqdm(test_id):
    testX = np.array([0]*len(terms_li))
    f = open('data/IRTM/'+str(doc)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
    tokens_all = dict(Counter(tokens_all))
    for key,value in tokens_all.items():
        if key in terms_li:
            testX[terms_li.index(key)] = int(value)

    test_X.append(testX)
        
test_X = np.array(test_X)
print(test_X.shape)

100%|██████████| 900/900 [00:07<00:00, 121.82it/s]

(900, 422)


In [100]:
# train_id.extend(trainid[1:])
# test_id
ans = MNB(input_X=test_X,prior_log_class=param1,log_prob_feature=param2,train=False,prob=False)
ans = ans.tolist()
ans

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 12,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 1,
 8,
 8,
 8,
 8,
 8,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 1,
 8,
 8,
 8,
 8,
 8,
 1,
 8,
 8,
 8,
 8,
 8,
 8,
 11,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 1,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 0,
 10,
 8,
 8,
 10,
 8,
 8,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8

In [112]:
print(ans)
ans2 = ans2.tolist()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 12, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 11, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 10, 8, 8, 10, 8, 8, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 9, 8, 8, 10, 9, 8, 8, 9, 9, 10, 10, 8, 8, 10, 9, 8, 10, 8, 9, 8, 0, 9, 8, 9, 8, 9, 10, 8, 9, 8, 8, 8, 0, 9, 9, 0, 8, 8, 9, 9, 0, 8, 8, 9, 0, 9, 0, 0, 8, 0, 8, 0, 0, 8, 9, 9, 9, 9, 9, 0, 9, 8, 0, 0, 9, 9, 0, 10, 8, 9, 8, 8

In [113]:
df_ans = pd.DataFrame(list(zip(test_id,ans2)),columns=['id','Value'])
df_ans.to_csv('output/MNB02_sk.csv',index=False)
df_ans

,id,Value
0,17,2
1,18,2
2,20,2
3,21,2
4,22,2
5,23,2
6,24,2
7,25,2
8,26,2
9,27,2


In [127]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [128]:
# Load iris data
iris = load_iris()

# Create features and target
X = iris.data
y = iris.target

# Convert to categorical data by converting data to integers
X = X.astype(int)

In [111]:
from sklearn.naive_bayes import *
clf = MultinomialNB()
clf.fit(train_X, train_Y)
ans2 = clf.predict(test_X)
ans2

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,  2,  2,  2,  2, 13,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  2,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  9,  9,  9,  9,  9,
        2,  9,  9,  9,  9,  9,  9, 12,  9,  9,  9,  9,  9,  9,  9,  9,  2,
        9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9